In [1]:
using Distributions, TimeIt, PyPlot, JLD, ProgressMeter, CSV
include("/home/postdoc/dsen/Desktop/Biometrika-code/zz_structures.jl")
include("/home/postdoc/dsen/Desktop/Biometrika-code/mbsampler.jl")

get_clustering (generic function with 1 method)

In [2]:
data = CSV.read("/xtmp/PDMP_data/risk_factors_cervical_cancer.csv")
name = names(data);
predictors = vcat(Array(2:25), Array(27:28));

In [3]:
for i in predictors
    present = map(x->parse(Float64,x),data[name[i]][data[name[i]] .!= "?"])
    present_mean = mean(present)
    absent = data[name[i]] .== "?"
    data[name[i]][absent] = string(present_mean)
    data[name[i]] = map(x->parse(Float64,x),data[name[i]])     
    print(i, " ")
end 
y = Int64.(data[name[29]]);
data = Float64.(convert(Array, data));

2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 27 28 

In [4]:
x = data[:,(sum(abs.(data),1) .!= 0)[1,:]];
X = sparse(x')
d, Nobs = size(X);
my_ll = ll_logistic_sp(X,y);
my_prior = gaussian_prior_nh(zeros(d),1*ones(d))
my_model = model(my_ll, my_prior)
σ02 = 1
root = find_root(my_model, rand(d));
gc()

In [5]:
β_0 = copy(root)
mb_size = 20

20

In [6]:
# Sub-sampling with weights and no CV:

gs = Array{mbsampler}(d)
gs[1] = umbsampler(Nobs, mb_size)
@showprogress for i in 2:d 
    nz_ind = X[i,:].nzind
    weights_het = spzeros(Nobs)
    weights_het[nz_ind] = abs.(X[i,nz_ind])./sum(abs.(X[i,nz_ind]))
    prob_het = (length(nz_ind)==length(X[i,:])) + 0.98*(length(nz_ind)!=length(X[i,:]))
    gs[i] = spwumbsampler(Nobs, mb_size, weights_het, prob_het)
end
gs_list_wt_noCV = mbsampler_list(d,gs);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:01


#### Run algorithms:

In [7]:
A = eye(d)
opf = projopf(A, 100)
opt = maxa_opt(5*10^7)
adapt_speed = "none"
L = 2;

In [8]:
mstate_wt_noCV = zz_state(d);
bb_ll_wt_noCV = build_bound(my_model.ll, gs_list_wt_noCV)
bb_pr_wt_noCV = build_bound(my_model.pr)
update_bound(bb_pr_wt_noCV, my_model.pr, mstate_wt_noCV)
update_bound(bb_ll_wt_noCV, my_model.ll, mstate_wt_noCV, gs_list_wt_noCV)
outp_wt_noCV = outputscheduler(deepcopy(opf),deepcopy(opt))

my_zz_sampler_wt_noCV = zz_sampler_decoupled(0, gs_list_wt_noCV, bb_pr_wt_noCV, bb_ll_wt_noCV, L, adapt_speed)
ZZ_sample_decoupled(my_model, outp_wt_noCV, my_zz_sampler_wt_noCV, mstate_wt_noCV)

filename  = "/xtmp/PDMP_data_revision/cervical_cancer/wt_noCV.jld"
save(filename, "xt_skeleton", outp_wt_noCV.opf.xi_skeleton, "bt_skeleton", outp_wt_noCV.opf.bt_skeleton)

10% attempts in 9.16 mins 
20% attempts in 18.41 mins 
30% attempts in 27.39 mins 
40% attempts in 35.17 mins 
50% attempts in 43.15 mins 
60% attempts in 51.16 mins 
70% attempts in 59.22 mins 
80% attempts in 67.34 mins 
90% attempts in 75.44 mins 
100% attempts in 83.24 mins 
